In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


In [2]:
#np.random.seed(404)
np.set_printoptions(precision=2, suppress=True)

In [3]:
# Read data
data = pd.read_excel('Modelling_Data_Phase_3.xlsx', sheet_name ='Seed', index_col=False) #change the sheet name according to the tabs to run the various cycles in the current Phase
data.head()

,Zr,Cu,Co,Fe,T,H2:CO,GHSV,STY,SCO2,SCH4,S
0,0.1010,0.478,0.213,0.208,260.0,2.0,24000.0,158.555963,0.080691,0.259485,0.340176
1,0.1010,0.202,0.480,0.217,260.0,2.0,24000.0,220.116161,0.057023,0.178532,0.235555
2,0.1170,0.421,0.104,0.360,260.0,2.0,24000.0,242.502579,0.100584,0.253891,0.354474
3,0.0966,0.201,0.230,0.465,260.0,2.0,24000.0,270.214203,0.109420,0.222736,0.332156
4,0.1280,0.112,0.401,0.359,260.0,2.0,24000.0,278.589571,0.100988,0.231617,0.332606


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Zr      98 non-null     float64
 1   Cu      98 non-null     float64
 2   Co      98 non-null     float64
 3   Fe      98 non-null     float64
 4   T       98 non-null     float64
 5   H2:CO   98 non-null     float64
 6   GHSV    98 non-null     float64
 7   STY     98 non-null     float64
 8   SCO2    98 non-null     float64
 9   SCH4    98 non-null     float64
 10  S       98 non-null     float64
dtypes: float64(11)
memory usage: 8.5 KB


In [37]:
# General stastistical data
data.describe()

,Zr,Cu,Co,Fe,T,H2:CO,GHSV,STY,SCO2,SCH4,S
count,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000
mean,0.133997,0.150402,0.196707,0.518860,267.878737,2.133737,42438.362361,415.434460,0.150048,0.249465,0.399513
std,0.121236,0.132859,0.111801,0.194090,13.708163,0.489743,23897.723771,282.898020,0.063784,0.041719,0.058100
min,0.048322,0.049333,0.049378,0.048463,233.900000,1.150000,10250.000000,13.552616,0.025419,0.177957,0.235555
25%,0.085497,0.085033,0.116681,0.359250,260.000000,1.942500,24000.000000,180.518892,0.099312,0.222674,0.359692
50%,0.101033,0.100679,0.189440,0.605000,262.250000,2.000000,31275.000000,346.229378,0.150435,0.242326,0.400826
75%,0.117170,0.124477,0.226046,0.651355,280.000000,2.445000,68912.500000,579.059711,0.196045,0.274746,0.435849
max,0.846036,0.663680,0.485278,0.806660,314.300000,3.150000,90000.000000,1098.461455,0.303248,0.397109,0.530177


In [38]:
# Define X and Y from the data

X = data[['Zr ', 'Cu ', 'Co ', 'Fe','T','H2:CO','GHSV']]
Y1 = data['STY']
Y2 = -1*data['S'] # S corresponds to combined undesired selectivity. We multiply by -1 for modeling essence, as we want to minimize this variable while maximize yield.
Y = pd.concat([Y1, Y2], axis=1)

In [39]:
#Check data for X or Y
Y

,STY,S
0,158.555963,-0.340176
1,220.116161,-0.235555
2,242.502579,-0.354474
3,270.214203,-0.332156
4,278.589571,-0.332606
...,...,...
93,178.646355,-0.320670
94,379.468383,-0.367808
95,540.070914,-0.348440
96,900.586297,-0.388893


In [40]:
# Feature normalization for X.

from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler(feature_range=(0,1))
mm.fit(np.array(X)[:,4:])

def scalex(x):
    x = np.array(x)
    x[:,4:] = mm.transform(x[:,4:])
    return x

def scalex_const(x_const):
    x_const = np.array(x_const)
    x_const = mm.transform(x_const)
    return x_const

X = scalex(X)

In [41]:
# Feature standardization for Y, applied to Y. Invert Y by multiplying with -1 to form a MAXIMIZATION problem as BO is MINIMIZATION by default

from sklearn.preprocessing import StandardScaler

stc = StandardScaler()
stc.fit(np.array(Y))

def scaley(y):
    y_scaled = -1*stc.transform(np.array(y))
    return y_scaled

def scaley_inv(y_scaled):
    y = stc.inverse_transform(np.array(-1*y_scaled))
    return y

Y = scaley(Y)

In [42]:
#Check the values of X or Y
X

array([[0.101, 0.478, 0.213, 0.208, 0.325, 0.425, 0.172],
       [0.101, 0.202, 0.48 , 0.217, 0.325, 0.425, 0.172],
       [0.117, 0.421, 0.104, 0.36 , 0.325, 0.425, 0.172],
       [0.097, 0.201, 0.23 , 0.465, 0.325, 0.425, 0.172],
       [0.128, 0.112, 0.401, 0.359, 0.325, 0.425, 0.172],
       [0.128, 0.107, 0.161, 0.605, 0.325, 0.425, 0.172],
       [0.624, 0.052, 0.054, 0.269, 0.325, 0.425, 0.172],
       [0.051, 0.351, 0.357, 0.241, 0.325, 0.425, 0.172],
       [0.244, 0.266, 0.211, 0.279, 0.325, 0.425, 0.172],
       [0.279, 0.05 , 0.343, 0.328, 0.325, 0.425, 0.172],
       [0.146, 0.074, 0.183, 0.597, 0.325, 0.425, 0.172],
       [0.082, 0.099, 0.214, 0.605, 0.325, 0.425, 0.172],
       [0.846, 0.056, 0.05 , 0.048, 0.325, 0.425, 0.172],
       [0.499, 0.394, 0.055, 0.052, 0.325, 0.425, 0.172],
       [0.514, 0.051, 0.383, 0.052, 0.325, 0.425, 0.172],
       [0.108, 0.056, 0.129, 0.707, 0.325, 0.425, 0.172],
       [0.075, 0.057, 0.061, 0.807, 0.325, 0.425, 0.172],
       [0.094,

In [13]:
# Import necesarry libraries for Gaussian process regression 

from gpflow.models import GPR
from gpflow.models import SVGP
from gpflow.likelihoods import Gaussian
from gpflow.optimizers import Scipy
from gpflow.kernels import SquaredExponential as SE, Constant as C, White as W, SharedIndependent as SI
from gpflow.inducing_variables import SharedIndependentInducingVariables as SIIV, InducingPoints as IP
from sklearn.metrics import r2_score, mean_squared_error

In [43]:
#Define the kernels using squared exponential. The dimentions of lengthscales must match the number of input features
#This is a multi objective task with 7 input features, where each feature correspond to the metal composition and reaction conditions

kernel = SE(lengthscales=[0.1,0.1,0.1,0.1,0.1,0.1,0.1])
#kernel = SE(lengthscales=7*[0.2])

# Gaussian process regression
gp_model = GPR((X, Y), kernel=kernel)

# Optimize the lengthscales
opt = Scipy()
opt.minimize(gp_model.training_loss, gp_model.trainable_variables)


  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 101.11957565406374
        x: [-2.890e-02  7.381e+02 -1.031e-01 -1.010e+00 -9.165e-01
             2.821e-01 -3.109e-01  1.402e+00 -2.847e+00]
      nit: 59
      jac: [ 1.361e-04 -1.352e-09 -2.951e-04  8.172e-04  4.811e-05
             3.287e-04 -1.686e-03  1.857e-04  1.756e-03]
     nfev: 70
     njev: 70
 hess_inv: <9x9 LbfgsInvHessProduct with dtype=float64>

In [44]:
from sklearn.metrics import r2_score, mean_squared_error

# After optimization, you can make predictions with the trained model
Y_pred_mean, _ = gp_model.predict_y(X)  # Predicted mean values

# Evaluate R^2 score
r2 = r2_score(Y, Y_pred_mean)

# Evaluate mean squared error (RMSE)
mse = mean_squared_error(Y, Y_pred_mean)
rmse = np.sqrt(mse)

# Print the evaluation metrics with limited decimal places

print("R^2 score: {:.2f}".format(r2))
print("Root Mean Squared Error: {:.2f}".format(rmse))

R^2 score: 0.96
Root Mean Squared Error: 0.19


In [45]:
# Check the optimized hyperparameters

optimized_lengthscales = gp_model.kernel.lengthscales.numpy()
print("Optimized Lengthscales:", optimized_lengthscales)

Optimized Lengthscales: [  0.679 738.064   0.643   0.311   0.336   0.844   0.55 ]


# Bayesian optimization

In [17]:
from trieste.space import LinearConstraint
from trieste.space import Box

# Define lower and upper bounds for metal fractions
Zr_lb = 0.1
Zr_ub = 0.1
Cu_lb = 0.05
Cu_ub = 0.2
Co_lb = 0.1
Co_ub = 0.3
Fe_lb = 0.4
Fe_ub = 0.8
T_lb = 250
T_ub = 300
H2_CO_lb = 1
H2_CO_ub = 4
GHSV_lb = 30000
GHSV_ub = 50000

const_lb = -10
const_ub = 10

const_mat = np.array([T_lb, H2_CO_lb, GHSV_lb, T_ub, H2_CO_ub, GHSV_ub]).reshape(2,3)
print(const_mat)
const_mat = scalex_const(const_mat)
print(const_mat)

T_lb = const_mat[0,0]
T_ub = const_mat[1,0]
H2_CO_lb = const_mat[0,1]
H2_CO_ub = const_mat[1,1]
GHSV_lb = const_mat[0,2]
GHSV_ub = const_mat[1,2]


# Define linear constraints. Apply lb and ub to the scalar product of the number vector and the feature vector

# Metal compositions + reaction conditions
constraints = [LinearConstraint(A=tf.constant
       ([[1, 1, 1, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], 
        [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1]]), 
                                
        lb=tf.constant([1, Zr_lb, Cu_lb, Co_lb, Fe_lb, T_lb, H2_CO_lb, GHSV_lb]), 
        ub=tf.constant([1, Zr_ub, Cu_ub, Co_ub, Fe_ub, T_ub, H2_CO_ub, GHSV_ub]))]

constrained_search_space = Box([0, 0, 0, 0, const_lb, const_lb, const_lb], 
                               [1, 1, 1, 1, const_ub, const_ub, const_ub], 
                               constraints=constraints)

[[  250     1 30000]
 [  300     4 50000]]
[[ 0.13 -0.07  0.25]
 [ 0.81  1.43  0.5 ]]


In [18]:
# Essential functions for formatting data

from trieste.data import Dataset

def observer(in_):
    in_ = tf.convert_to_tensor(in_)
    out_, _ = gp_model.predict_y(in_)
    out_ = tf.convert_to_tensor(out_)
    return Dataset(in_, out_)

def initial_data(in_, out_):
    in_ = tf.convert_to_tensor(in_)
    out_ = tf.convert_to_tensor(out_)
    return Dataset(in_, out_)

In [19]:
# Import necessary libraries to build model

from trieste.models.gpflow import GaussianProcessRegression
from trieste.bayesian_optimizer import BayesianOptimizer
from trieste.acquisition.rule import EfficientGlobalOptimization
from trieste.acquisition.function import Fantasizer
from trieste.acquisition import LocalPenalization
from trieste.acquisition.function import ExpectedHypervolumeImprovement
from trieste.acquisition.function import ExpectedImprovement
from trieste.acquisition.function import PredictiveVariance

model = GaussianProcessRegression(gp_model, num_kernel_samples=10)

# Acquisition functions and rule

ei = ExpectedImprovement(constrained_search_space)
rule_ei = EfficientGlobalOptimization(builder=ei)

pv = PredictiveVariance()
rule_pv = EfficientGlobalOptimization(builder=pv)

ehvi = ExpectedHypervolumeImprovement()
rule_ehvi = EfficientGlobalOptimization(builder=ehvi)

# Bayesian optimizer
bo = BayesianOptimizer(observer, constrained_search_space)


In [20]:
# Run the Bayesian optimizer for multi objective

batch_size = 10 # This number is user defined and determines the number of recommendation made by the BO. Typically 5-10 generations yield good results. For the project, we used a batch of 30. 

#We use the rule_ehvi or rule_pv as parmaters when runnig the mutli-objective to enable fair trade-off between exploitation or exploration campaigns, respectively.

bo_result = bo.optimize(batch_size, initial_data(X, Y), model, rule_ehvi, track_state = False, fit_initial_model=False)

C:\Users\msuvarna\Anaconda3\envs\HAS\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Optimization completed without errors


In [22]:
# Get results from the Bayesian optimizer

bo_initial_data = bo_result.try_get_final_dataset()
bo_X = bo_result.try_get_final_dataset().query_points.numpy()[-batch_size:,:]

bo_X[:,4:] = mm.inverse_transform(bo_X[:,4:]) 

bo_Y = bo_result.try_get_final_dataset().observations.numpy()[-batch_size:,:]
np.set_printoptions(precision=3, suppress=True)

result=(np.concatenate((bo_X, scaley_inv(bo_Y)), axis=1))


In [23]:
# Create dataframe with results 

dfresult = pd.DataFrame(result, columns = ['Zr','Cu','Co','Fe','T','H2:CO','GHSV','STY','SCO2+CH4'])
dfresult = dfresult.round(2)
dfresult

,Zr,Cu,Co,Fe,T,H2:CO,GHSV,STY,SCO2+CH4
0,0.1,0.12,0.3,0.48,261.66,1.79,50000.0,501.08,-0.30
1,0.1,0.05,0.3,0.55,265.81,1.86,50000.0,602.41,-0.33
2,0.1,0.20,0.3,0.40,259.22,1.72,50000.0,400.09,-0.29
3,0.1,0.05,0.3,0.55,250.00,1.91,50000.0,393.36,-0.29


In [ ]:
#The above dataframe is the output of the current campiagn recommending catalyst composition worth investigation and its predicted yield and selectivity
#In the study we performed these experimental recommendation and measured the actual yield and selectiviy.